In [2]:
import os,sys,glob
from matplotlib import colors as mcolors
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import seaborn as sns
from itertools import combinations
from sklearn.metrics import mean_squared_error
from scipy import stats

In [15]:
### subjects
def collectSubjectData(topPath,dataPath,groups,subjects,colors):

    # set up variables
    data_columns = ['subjectID','classID','colors']
    data =  pd.DataFrame([],columns=data_columns)

    # populate structure
    data['subjectID'] = [ f for g in groups for f in subjects[g] ]
    data['classID'] = [ g for g in groups for f in range(len(subjects[g]))]
    data['colors'] = [ colors[c] for c in colors for f in subjects[c]]

    # output data structure for records and any further analyses
    if not os.path.exists(dataPath):
        os.mkdir(dataPath)

    data.to_csv(dataPath+'subjects.csv',index=False)

    return data

### color dictionary
def createColorDictionary(data,measure,colorPalette):

    keys = data[measure].unique()
    values = sns.color_palette(colorPalette,len(keys))
    values = values.as_hex()

    colors_dict = dict(zip(keys,values))

    return colors_dict

### load parcellation stats data 
### load data 
def collectData(datatype,datatype_tags,tags,filename,subjects_data,colors,outPath):

    import requests
    import pandas as pd

    # grab path and data objects
    objects = requests.get('https://brainlife.io/api/warehouse/secondary/list/%s'%os.environ['PROJECT_ID']).json()

    # subjects and paths
    subjects = []
    paths = []

    # set up output
    data = pd.DataFrame()

    # loop through objects
    for obj in objects:
        if obj['datatype']['name'] == datatype:
            if datatype_tags in obj['output']['datatype_tags']:
    #                 if tags in obj['output']['tags']:
                if set(tags).issubset(obj['output']['tags']):

                    subjects = np.append(subjects,obj['output']['meta']['subject'])
                    paths = np.append(paths,"input/"+obj["path"]+"/"+filename)
                elif '!' in str(tags):
                    tag = [ f for f in tags if '!' in str(f) ]
                    tag_drop = [ f for f in tags if f not in tag ]
                    if not set([ f.replace('!','') for f in tag]).issubset(obj['output']['tags']):
                        if set(tag_drop).issubset(obj['output']['tags']):
                            subjects = np.append(subjects,obj['output']['meta']['subject'])
                            paths = np.append(paths,"input/"+obj["path"]+"/"+filename)

    paths = [y for _,y in sorted(zip(subjects,paths))]
    subjects = [x for x,_ in sorted(zip(subjects,paths))]

    for i in paths:
        if '.json.gz' in filename:
            tmpdata = pd.read_json(i,orient='index').reset_index(drop=True)
            tmpdata['subjectID'] = [ str(subjects[f]) for f in range(len(subjects)) if i == paths[f]]
            tmpdata = pd.merge(tmpdata,subjects_data)
        else:
            tmpdata = pd.read_csv(i)
        if tmpdata.subjectID.dtypes != 'object':
            tmpdata['subjectID'] = [ str(int(np.float(f))) for f in tmpdata.subjectID ]
    #         if 'classID' in tmpdata.keys():
    #             tmpdata = pd.merge(tmpdata,subjects_data,on=['subjectID','classID'])
    #         else:
    #             tmpdata = pd.merge(tmpdata,subjects_data,on='subjectID')
        data = data.append(tmpdata,ignore_index=True)

    # replace empty spaces with nans
    data = data.replace(r'^\s+$', np.nan, regex=True)

    # drop duplicates
#     data = data.drop_duplicates('subjectID')

    # output data structure for records and any further analyses
    # subjects.csv
    data.to_csv(outPath,index=False)

    return data

def collectNetworkData(datatype,datatype_tags,tags,corr_filename,labels_filename,subjects_data,colors,outPath):

    import requests
    import pandas as pd

    # grab path and data objects
    objects = requests.get('https://brainlife.io/api/warehouse/secondary/list/%s'%os.environ['PROJECT_ID']).json()

    # subjects and paths
    subjects = []
    csv_paths = []
    label_paths = []

    # set up output
    data = pd.DataFrame()

    # loop through objects
    for obj in objects:
        if obj['datatype']['name'] == datatype:
            if datatype_tags in obj['output']['datatype_tags']:
                if tags in obj['output']['tags']:
                    subjects = np.append(subjects,obj['output']['meta']['subject'])
                    csv_paths = np.append(csv_paths,"input/"+obj["path"]+"/"+corr_filename)
                    label_paths = np.append(label_paths,"input/"+obj["path"]+"/"+labels_filename)

    # sort paths by subject order
    subjects = [x for _,x in sorted(zip(subjects,subjects))]
    csv_paths = [x for _,x in sorted(zip(subjects,csv_paths))]
    label_paths = [x for _,x in sorted(zip(subjects,label_paths))]

    for i in range(len(csv_paths)):
        tmplabel = pd.read_json(label_paths[i])
        label_names = [ f for f in tmplabel['name'] if f not in ['self-loop'] ]
        tmpdata = pd.read_csv(csv_paths[i],names=label_names)
        tmpdata.index = label_names
        tmpdata['subjectID'] = [ subjects[i] for f in range(len(tmpdata)) ]
        if tmpdata.subjectID.dtypes != 'object':
            tmpdata['subjectID'] = [ str(int(np.float(f))) for f in tmpdata.subjectID ]
        if 'classID' in tmpdata.keys():
            tmpdata = pd.merge(tmpdata,subjects_data,on=['subjectID','classID'],right_index=True)
        else:
            tmpdata = pd.merge(tmpdata,subjects_data,on='subjectID',right_index=True)
        data = data.append(tmpdata)

    # replace empty spaces with nans
    data = data.replace(r'^\s+$', np.nan, regex=True)

    # output data structure for records and any further analyses
    # subjects.csv
    data.to_csv(outPath)

    return data

### cut nodes
def cutNodes(data,num_nodes,dataPath,foldername,savename):

    # identify inner n nodes based on num_nodes input
    total_nodes = len(data['nodeID'].unique())
    cut_nodes = int((total_nodes - num_nodes) / 2)

    # remove cut_nodes from dataframe
    data = data[data['nodeID'].between((cut_nodes)+1,(num_nodes+cut_nodes))]

    # replace empty spaces with nans
    data = data.replace(r'^\s+$', np.nan, regex=True)

    # output data structure for records and any further analyses
    if not os.path.exists(dataPath):
        os.mkdir(dataPath)

    data.to_csv(dataPath+'/'+foldername+'-'+savename+'.csv',index=False)

    return data

def computeMeanData(dataPath,data,outname):

    # make mean data frame
    data_mean =  data.groupby(['subjectID','classID','structureID']).mean().reset_index()
    data_mean['nodeID'] = [ 1 for f in range(len(data_mean['nodeID'])) ]

    # output data structure for records and any further analyses
    if not os.path.exists(dataPath):
        os.mkdir(dataPath)

    data_mean.to_csv(dataPath+outname+'.csv',index=False)

    return data_mean

### rank order effect size calculator
def computeRankOrderEffectSize(groups,subjects,tissue,measures,stat,measures_to_average,data_dir):

    comparison_array = list(combinations(groups,2)) # 2 x 2 array; 2 different comparisons, with two pairs per comparison. comparison_array[0] = ("run_1","run_2")
    es = {}
    roes = {}

    # compute effect size
    for compar in comparison_array:
        es[compar[0]+"_"+compar[1]] = pd.DataFrame([])
        tmp = pd.DataFrame([])
        tmp['structureID'] = stat['structureID'].unique()
        for m in measures:
            diff = stat[['structureID',m]][stat['classID'].str.contains(compar[0])].groupby('structureID').mean() - stat[['structureID',m]][stat['classID'].str.contains(compar[1])].groupby('structureID').mean()
            pooled_var = (np.sqrt((stat[['structureID',m]][stat['classID'].str.contains(compar[0])].groupby('structureID').std() ** 2 + stat[['structureID',m]][stat['classID'].str.contains(compar[1])].groupby('structureID').std() ** 2) / 2))
            effectSize = diff / pooled_var
            tmp[m+"_effect_size"] = list(effectSize[m])
        tmp.to_csv(data_dir+tissue+"_effect_sizes_"+compar[0]+"_"+compar[1]+".csv",index=False)
        es[compar[0]+"_"+compar[1]] = pd.concat([es[compar[0]+"_"+compar[1]],tmp],ignore_index=True)

    # rank order structures
    for ma in measures_to_average:
        if ma == ['ad','fa','md','rd','ga','ak','mk','rk']:
            model = 'tensor'
        elif ma == ['ndi','isovf','odi']:
            model = 'noddi'
        else:
            model = ma

        tmpdata = pd.DataFrame([])
        tmpdata['structureID'] = stat['structureID'].unique()
        for compar in comparison_array:
            if model == 'tensor':
                tmpdata[compar[0]+"_"+compar[1]+"_"+model+"_average_effect_size"] = es[compar[0]+"_"+compar[1]][['ad_effect_size','fa_effect_size','md_effect_size','rd_effect_size']].abs().mean(axis=1).tolist()
            elif model == 'noddi':
                tmpdata[compar[0]+"_"+compar[1]+"_"+model+"_average_effect_size"] = es[compar[0]+"_"+compar[1]][['ndi_effect_size','isovf_effect_size','odi_effect_size']].abs().mean(axis=1).tolist()
            else:
                tmpdata[compar[0]+"_"+compar[1]+"_"+model+"_average_effect_size"] = es[compar[0]+"_"+compar[1]][[ma+'_effect_size']].abs().mean(axis=1).tolist()

        tmpdata[model+"_average_effect_size"] =  tmpdata.mean(axis=1).tolist()
        tmpdata.to_csv(data_dir+model+"_average_"+tissue+"_effect_sizes.csv",index=False)
        roes[model] = tmpdata.sort_values(by=model+"_average_effect_size")['structureID'].tolist()

    return roes

def combineCorticalSubcortical(dataPath,corticalData,subcorticalData):

    # remove unnecessary columns
    corticalData = corticalData.drop(columns=['snr','thickness'])
    subcorticalData = subcorticalData.drop(columns=['parcID','number_of_voxels'])

    # merge data frames
    data = pd.concat([corticalData,subcorticalData],sort=False)

    # output data structure for records and any further analyses
    if not os.path.exists(dataPath):
        os.mkdir(dataPath)

    data.to_csv(dataPath+'graymatter_nodes.csv',index=False)

    # identify gray matter names
    graymatter_names = list(data['structureID'].unique())

    # output track names
    if not os.path.exists(dataPath):
        os.mkdir(dataPath)

    with open((dataPath+'graymatter_list.json'),'w') as gm_listf:
        json.dump(graymatter_names,gm_listf)

    return [graymatter_names,data]

def computeDistance(x,y,metric):

    from sklearn.metrics.pairwise import euclidean_distances
    from scipy.stats import wasserstein_distance

    if metric == 'euclidean':
        dist = euclidean_distances([x,y])[0][1]
    else:
        dist = wasserstein_distance(x,y)
        
    return dist

def computeReferences(x,groupby_measures,index_measure,diff_measures):
    
    references_mean = x.groupby(groupby_measures).mean().reset_index(index_measure)
    references_sd = x.groupby(groupby_measures).std().reset_index(index_measure)
    references_sd[diff_measures] = references_sd[diff_measures] * 2
    
    return references_mean, references_sd

def createDistanceDataframe(data,structures,groupby_measure,measures,dist_metric):
    
    dist = []
    subj = []
    meas = []
    struc = []

    for i in structures:
        print(i)
        subj_data = data.loc[data['structureID'] == i]
        references_data = computeReferences(subj_data,groupby_measure,groupby_measure,measures)
        for m in measures:
            for s in subj_data.subjectID.unique():
                x = list(subj_data.loc[subj_data['subjectID'] == s][m].values.tolist())
                y = list(references_data[0][m].values.tolist())
                dist = np.append(dist,computeDistance(x,y,dist_metric))
                subj = np.append(subj,s)
                meas = np.append(meas,m)
                struc = np.append(struc,i)

    dist_dataframe = pd.DataFrame()
    dist_dataframe['subjectID'] = subj
    dist_dataframe['structureID'] = struc
    dist_dataframe['measures'] = meas
    dist_dataframe['distance'] = dist
    
    return dist_dataframe

def buildReferenceData(data,outliers,profile):
    
    reference_data = pd.DataFrame()
    
    for s in outliers.structureID.unique():
        for m in outliers.measures.unique():
            if profile:
                tmpdata = data.loc[data['structureID'] == s].loc[~data['subjectID'].isin(outliers.loc[outliers['structureID'] == s].loc[outliers['measures'] == m].subjectID.unique())][['structureID','subjectID','nodeID',m]].reset_index(drop=True)
            else:
                tmpdata = data.loc[data['structureID'] == s].loc[~data['subjectID'].isin(outliers.loc[outliers['structureID'] == s].loc[outliers['measures'] == m].subjectID.unique())][['structureID','subjectID',m]].reset_index(drop=True)
            reference_data = pd.concat([reference_data,tmpdata])
    if not profile:
        reference_data = reference_data.groupby(['structureID','subjectID']).mean().reset_index()
    
    return reference_data

def computeOutliers(distances,threshold):
    
    outliers = pd.DataFrame()
    
    for i in distances.structureID.unique():
        for m in distances.measures.unique():
            tmpdata = distances.loc[distances['structureID'] == i].loc[distances['measures'] == m]
            outliers = pd.concat([outliers,tmpdata[tmpdata['distance'] > np.percentile(tmpdata['distance'],threshold)]])
            
    return outliers

def outlierDetection(data,structures,groupby_measure,measures,threshold,dist_metric,build_outliers):
    
    import numpy as np, pandas as pd

    outliers_subjects = []
    outliers_structures = []
    outliers_measures = []
    outliers_metrics = []

    distances = createDistanceDataframe(data,structures,groupby_measure,measures,dist_metric)
    outliers_dataframe = computeOutliers(distances,threshold)
    
    if build_outliers:
        if 'nodeID' in data.columns:
            reference_dataframe = buildReferenceData(data,outliers_dataframe,True)
        else:
            reference_dataframe = buildReferenceData(data,outliers_dataframe,False)
    else:
        reference_dataframe = []
        
    return distances, outliers_dataframe, reference_dataframe

def profileFlipCheck(data,subjects,structures,test_measure,flip_measures,dist_metric,threshold,outPath):
    
    flipped_subjects = []
    flipped_structures = []
    distance = []
    flipped_distance = []
    
    for i in structures:
        print(i)
        struc_data = data.loc[data['structureID'] == i]
        references_data = computeReferences(struc_data,'nodeID','nodeID',flip_measures)
        differences = []
        dist = []
        dist_flipped = []

        for s in subjects:
            subj_data = struc_data.loc[data['subjectID'] == s]
            x = list(subj_data[test_measure].values.tolist())
            y = list(references_data[0][test_measure].values.tolist())
            dist = np.append(dist,computeDistance(x,y,dist_metric))
            dist_flipped = np.append(dist_flipped,computeDistance(list(np.flip(x)),y,dist_metric))
            differences =  np.append(differences,(dist[-1]-dist_flipped[-1]))
        
        percentile_threshold = np.percentile(differences,threshold)
#         print(percentile_threshold)
        for m in range(len(differences)):
            if differences[m] > 0 and differences[m] > percentile_threshold:
#             if differences[m] > percentile_threshold:
#                 print(subjects[m])
                flipped_subjects = np.append(flipped_subjects,subjects[m])
                flipped_structures = np.append(flipped_structures,i)
                distance = np.append(distance,dist[m])
                flipped_distance = np.append(flipped_distance,dist_flipped[m])
    
    output_summary = pd.DataFrame()
    output_summary['flipped_subjects'] = flipped_subjects
    output_summary['flipped_structures'] = flipped_structures
    output_summary['distance'] = distance
    output_summary['flipped_distance'] = flipped_distance
    
    if outPath:
        output_summary.to_csv(outPath+'_flipped_profiles.csv',index=False)
    
    return output_summary

### scatter plot related scripts
# groups data by input measure and computes mean for each value in that column. x_stat is a pd dataframe, with each row being a single value, and each column being a different ID value or measure
def averageWithinColumn(x_stat,y_stat,x_measure,y_measure,measure):

    X = x_stat.groupby(measure).mean()[x_measure].tolist()
    Y = y_stat.groupby(measure).mean()[y_measure].tolist()

    return X,Y

# groups data by input measure and creates an array by appending data into x and y arrays. x_stat and y_stat are pd dataframes, with each row being a single value, and each column being a different ID value or measure
# designed for test retest. x_stat and y_stat should have the same number of rows. but more importantly, should correspond to the same source (i.e. subject)
# can be same pd.dataframe, but indexing of specific subject groups
def appendWithinColumn(x_stat,y_stat,x_measure,y_measure,measure):

    X,Y = [np.array([]),np.array([])]
    for i in range(len(x_stat[measure].unique())):
        x = x_stat[x_stat[measure] == x_stat[measure].unique()[i]][x_measure]
        y = y_stat[y_stat[measure] == y_stat[measure].unique()[i]][y_measure]

        if np.isnan(x).any() or np.isnan(y).any():
            print("skipping %s due to nan" %x_stat[measure].unique()[i])
        else:
            # checks to make sure the same data
            if len(x) == len(y):
                X = np.append(X,x)
                Y = np.append(Y,y)

    return X,Y

# unravels networks. x_stat and y_stat should be S x M, where S is the number of subjects and M is the adjacency matrix for that subject
def ravelNetwork(x_stat,y_stat):

    import numpy as np

    X = np.ravel(x_stat).tolist()
    Y = np.ravel(y_stat).tolist()

    return X,Y

# unravels nonnetwork data. x_stat and y_stat should be pd dataframes. x_measure and y_measure are the measure to unrvavel. 
# designed for test retest. x_stat and y_stat should have the same number of rows. but more importantly, should correspond to the same source (i.e. subject)
# can be same pd.dataframe, but indexing of specific subject groups
def ravelNonNetwork(x_stat,y_stat,x_measure,y_measure):

    X = x_stat[x_measure].to_list()
    Y = y_stat[y_measure].to_list()

    return X,Y

# wrapper function to call either of the above scripts based on user input
def setupData(x_data,y_data,x_measure,y_measure,ravelAverageAppend,isnetwork,measure):

    x_stat = x_data
    y_stat = y_data

    if ravelAverageAppend == 'average':
        X,Y = averageWithinColumn(x_stat,y_stat,x_measure,y_measure,measure)
    elif ravelAverageAppend == 'append':
        X,Y = appendWithinColumn(x_stat,y_stat,x_measure,y_measure,measure)
    elif ravelAverageAppend == 'ravel':
        if isnetwork == True:
            X,Y = ravelNetwork(x_stat,y_stat)
        else:
            X,Y = ravelNonNetwork(x_stat,y_stat,x_measure,y_measure)

    return x_stat,y_stat,X,Y

# function to shuffle data and colors
def shuffleDataAlg(X,Y,hues):

    from sklearn.utils import shuffle

    X,Y,hues = shuffle(X,Y,hues)

    return X,Y,hues

# simple display or figure save function
def saveOrShowImg(dir_out,x_measure,y_measure,img_name):
    import os,sys 
    import matplotlib.pyplot as plt
    import warnings
    
    with warnings.catch_warnings():
        # this will suppress all warnings in this block
        warnings.simplefilter("ignore")
 
        # save or show plot
        if dir_out:
            if not os.path.exists(dir_out):
                os.mkdir(dir_out)

            if x_measure == y_measure:
                img_name_eps = img_name+'_'+x_measure+'.eps'
                img_name_png = img_name+'_'+x_measure+'.png'
                img_name_svg = img_name+'_'+x_measure+'.svg'
            else:
                img_name_eps = img_name+'_'+x_measure+'_vs_'+y_measure+'.eps'
                img_name_png = img_name+'_'+x_measure+'_vs_'+y_measure+'.png'
                img_name_svg = img_name+'_'+x_measure+'_vs_'+y_measure+'.svg'

            plt.savefig(os.path.join(dir_out, img_name_eps),transparent=True)
            plt.savefig(os.path.join(dir_out, img_name_png))     
    #         plt.savefig(os.path.join(dir_out, img_name_svg))
        else:
            plt.show()

        plt.close()
    
# uses seaborn's relplot function to plot data for each unique value in a column of a pandas dataframe (ex. subjects, structureID). useful for supplementary figures or sanity checking or preliminary results
# column measure is the measure within which each unique value will have its own plot. hue_measure is the column to use for coloring the data. column_wrap is how many panels you want per row
# trendline, depending on user input, can either be the linear regression between x_data[x_measure] and y_data[y_measure] or the line of equality
# dir_out and img_name are the directory where the figures should be saved and the name for the image. will save .eps and .png
# if want to view plot instead of save, set dir_out=""
def relplotScatter(x_data,y_data,x_measure,y_measure,column_measure,hue_measure,column_wrap,trendline,dir_out,img_name):

    import os,sys
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import mean_squared_error

    # grab data: CANNOT BE AVERAGE
    [x_stat,y_stat,X,Y] = setupData(x_data,y_data,x_measure,y_measure,'ravel',False,hue_measure)

    p = sns.relplot(x=X,y=Y,col=x_stat[column_measure],hue=x_stat[hue_measure],kind="scatter",s=100,col_wrap=column_wrap)

    # setting counter. looping through axes to add important info and regression lines
    i = 0
    for ax in p.axes.flat:
        x_lim,y_lim = [ax.get_xlim(),ax.get_ylim()]

        if trendline == 'equality':
            ax.plot(x_lim,y_lim,ls="--",c='k')
        elif trendline == 'linreg':
            m,b = np.polyfit(p.data[p.data[column_measure] == x_stat[column_measure].unique()[i]]['x'],p.data[p.data[column_measure] == y_stat[column_measure].unique()[i]]['y'],1)
            ax.plot(ax.get_xticks(),m*ax.get_xticks() + b)
            plt.text(0.1,0.7,'y = %s x + %s' %(str(np.round(m,4)),str(np.round(b,4))),fontsize=12,verticalalignment="top",horizontalalignment="left",transform=ax.transAxes)

        ax.set_xlim(x_lim)
        ax.set_ylim(y_lim)
        ax.set_xlabel(x_measure)
        ax.set_ylabel(y_measure)

        # compute correlation for each subject and add to plots
        corr = np.corrcoef(p.data[p.data[column_measure] == x_stat[column_measure].unique()[i]]['x'],p.data[p.data[column_measure] == y_stat[column_measure].unique()[i]]['y'])[1][0]
        plt.text(0.1,0.9,'r = %s' %str(np.round(corr,4)),fontsize=12,verticalalignment="top",horizontalalignment="left",transform=ax.transAxes)

        # compute rmse for each subject and add to plots
        rmse = np.sqrt(mean_squared_error(p.data[p.data[column_measure] == x_stat[column_measure].unique()[i]]['x'],p.data[p.data[column_measure] == y_stat[column_measure].unique()[i]]['y']))
        plt.text(0.1,0.8,'rmse = %s' %str(np.round(rmse,4)),fontsize=12,verticalalignment="top",horizontalalignment="left",transform=ax.transAxes)

        # update counter
        i = i+1

    # save image or show image
    saveOrShowImg(dir_out,x_measure,y_measure,img_name)

# uses seaborn's scatter function to plot data from x_data[x_measure] and y_data[y_measure]. useful for publication worthy figure
# column measure is the measure within which data will be summarized. hue_measure is the column to use for coloring the data. 
# ravelAverageAppend is a string value of either 'append' to use the append function, 'ravel' to use the ravel function, or 'average' to use the average function
# trendline, depending on user input, can either be the linear regression between x_data[x_measure] and y_data[y_measure] or the line of equality
# dir_out and img_name are the directory where the figures should be saved and the name for the image. will save .eps and .png
# if want to view plot instead of save, set dir_out=""
def singleplotScatter(colors_dict,x_data,y_data,x_measure,y_measure,logX,column_measure,hue_measure,ravelAverageAppend,trendline,shuffleData,dir_out,img_name):

    import os,sys
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import mean_squared_error

    # grab data
    [x_stat,y_stat,X,Y] = setupData(x_data,y_data,x_measure,y_measure,ravelAverageAppend,False,column_measure)
    colors = sns.color_palette('colorblind',len(x_stat[hue_measure]))

    if ravelAverageAppend == 'average':
        if isinstance(x_stat[hue_measure].unique()[0],str):
            hues = x_stat[hue_measure].unique().tolist()
        else:
            hues = x_stat.groupby(column_measure).mean()[hue_measure].tolist()
    else:
        hues = list(x_stat[hue_measure])

    if shuffleData == True:
        X,Y,hues = shuffleDataAlg(X,Y,hues)

    if logX == True:
        X = np.log10(X)

    if colors_dict:
        p = sns.scatterplot(x=X,y=Y,hue=hues,s=100,palette=colors_dict,legend=False)
    else:
        p = sns.scatterplot(x=X,y=Y,hue=hues,s=100)					

    # set x and ylimits, plot line of equality, and legend
    if x_measure == y_measure:
        p.axes.axis('square')
        y_ticks = p.axes.get_yticks()
        p.axes.set_xticks(y_ticks)
        p.axes.set_yticks(p.axes.get_xticks())
        p.axes.set_ylim(p.axes.get_xlim())
        p.axes.set_xlim(p.axes.get_xlim())

    x_lim,y_lim = [p.axes.get_xlim(),p.axes.get_ylim()]

    # trendline: either equality or linear regression
    if trendline == 'equality':
        p.plot(x_lim,y_lim,ls="--",c='k')
    elif trendline == 'linreg':
        m,b = np.polyfit(X,Y,1)
        p.plot(p.get_xticks(),m*p.get_xticks() + b,c='k')
        plt.text(0.1,0.7,'y = %s x + %s' %(str(np.round(m,4)),str(np.round(b,4))),fontsize=16,verticalalignment="top",horizontalalignment="left",transform=p.axes.transAxes)
        ax = plt.gca()
        ax.get_legend().remove()

    elif trendline == 'groupreg':
        for g in range(len(groups)):
            if stat_name == 'volume':
                slope, intercept, r_value, p_value, std_err = stats.linregress(np.log10(stat[['structureID',stat_name]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[stat_name]),stat[['structureID',dm]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[dm])
                ax = sns.regplot(x=np.log10(stat[['structureID',stat_name]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[stat_name]),y=stat[['structureID',dm]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[dm],color=colors[groups[g]],scatter=True,line_kws={'label':"y={0:.5f}x+{1:.4f}".format(slope,intercept)})

            else:
                slope, intercept, r_value, p_value, std_err = stats.linregress(stat[['structureID',stat_name]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[stat_name],stat[['structureID',dm]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[dm])
                ax = sns.regplot(x=stat[['structureID',stat_name]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[stat_name],y=stat[['structureID',dm]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[dm],color=colors[groups[g]],scatter=True,line_kws={'label':"y={0:.5f}x+{1:.4f}".format(slope,intercept)})

            ax.legend()


    # compute correlation for each subject and add to plots
    corr = np.corrcoef(X,Y)[1][0]
    plt.text(0.1,0.9,'r = %s' %str(np.round(corr,4)),fontsize=16,verticalalignment="top",horizontalalignment="left",transform=p.axes.transAxes)

    # compute rmse for each subject and add to plots
    rmse = np.sqrt(mean_squared_error(X,Y))
    plt.text(0.1,0.8,'rmse = %s' %str(np.round(rmse,4)),fontsize=16,verticalalignment="top",horizontalalignment="left",transform=p.axes.transAxes)

    # set title and x and y labels
    plt.title('%s vs %s' %(x_measure,y_measure),fontsize=20)
    plt.xlabel(x_measure,fontsize=18)
    plt.ylabel(y_measure,fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # remove top and right spines from plot
    p.axes.spines["top"].set_visible(False)
    p.axes.spines["right"].set_visible(False)

    # save image or show image
    saveOrShowImg(dir_out,x_measure,y_measure,img_name)

# uses seaborn's scatter function to plot data from x_data[x_measure] and y_data[y_measure] for network correlations. useful for publication worthy figure
# column measure is the measure within which data will be summarized.
# ravelAverageAppend is a string value of either 'append' to use the append function, 'ravel' to use the ravel function, or 'average' to use the average function
# trendline, depending on user input, can either be the linear regression between x_data[x_measure] and y_data[y_measure] or the line of equality
# dir_out and img_name are the directory where the figures should be saved and the name for the image. will save .eps and .png
# if want to view plot instead of save, set dir_out=""
def networkScatter(colors_dict,hues,groups,subjects,x_data,y_data,network_measure,shuffleData,trendline,dir_out,img_name):

    import os,sys
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import mean_squared_error

    # generate new figure for each
    p = plt.figure()

    # grab data
#     [x_stat,y_stat,X,Y] = setupData(x_data,y_data,"","","ravel",True,"")

    # additional network setup
    # hues = sns.color_palette(colormap,len(X))
    # hues = hues.as_hex()
    # keys = [ i for i in range(len(X)) ]
    # colors_dict = dict(zip(hues,hues))

#     if shuffleData == True:
#         X,Y,hues = shuffleDataAlg(X,Y,hues)

    # if colors_dict:
        # p = sns.scatterplot(x=X,y=Y,hue=hues,s=100,palette=colors_dict,legend=False)
    # else:
    
    p = sns.scatterplot(x=x_data,y=y_data,s=100,palette=colors_dict,legend=False)

    # set x and ylimits, plot line of equality, and legend
    p.axes.axis('square')
    y_ticks = p.axes.get_yticks()
    p.axes.set_xticks(y_ticks)
    p.axes.set_yticks(p.axes.get_xticks())
    p.axes.set_ylim(p.axes.get_xlim())
    p.axes.set_xlim(p.axes.get_xlim())

    x_lim,y_lim = [p.axes.get_xlim(),p.axes.get_ylim()]

    # trendline: either equality or linear regression
    if trendline == 'equality':
        p.plot(x_lim,y_lim,ls="--",c='k')
    elif trendline == 'linreg':
        m,b = np.polyfit(x_data,y_data,1)
        p.plot(p.get_xticks(),m*p.get_xticks() + b,c='k')
        plt.text(0.1,0.7,'y = %s x + %s' %(str(np.round(m,4)),str(np.round(b,4))),fontsize=16,verticalalignment="top",horizontalalignment="left",transform=p.axes.transAxes)

    # compute correlation for each subject and add to plots
    corr = np.corrcoef(x_data,y_data)[1][0]
    plt.text(0.1,0.9,'r = %s' %str(np.round(corr,4)),fontsize=16,verticalalignment="top",horizontalalignment="left",transform=p.axes.transAxes)

    # compute rmse for each subject and add to plots
    rmse = np.sqrt(mean_squared_error(x_data,y_data))
    plt.text(0.1,0.8,'rmse = %s' %str(np.round(rmse,4)),fontsize=16,verticalalignment="top",horizontalalignment="left",transform=p.axes.transAxes)

    # set title and x and y labels
    plt.title('%s %s vs %s' %(network_measure,groups[0],groups[1]),fontsize=20)
    plt.xlabel(groups[0],fontsize=18)
    plt.ylabel(groups[1],fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # remove top and right spines from plot
    p.axes.spines["top"].set_visible(False)
    p.axes.spines["right"].set_visible(False)

    # save image or show image
    saveOrShowImg(dir_out,network_measure+'_'+groups[0],network_measure+'_'+groups[1],img_name)

# uses matplotlib.pyplot's hist2d function to plot data from x_data[x_measure] and y_data[y_measure]. useful for supplementary figure or debugging or publication worthy figure
# column measure is the measure within which data will be summarized. hue_measure is the column to use for coloring the data. 
# ravelAverageAppend is a string value of either 'append' to use the append function, 'ravel' to use the ravel function, or 'average' to use the average function
# trendline, depending on user input, can either be the linear regression between x_data[x_measure] and y_data[y_measure] or the line of equality
# dir_out and img_name are the directory where the figures should be saved and the name for the image. will save .eps and .png
# if want to view plot instead of save, set dir_out=""
def plot2dHist(x_data,y_data,x_measure,y_measure,column_measure,hue_measure,ravelAverageAppend,trendline,shuffleData,dir_out,img_name):

    import os,sys
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import mean_squared_error

    # grab data
    [x_stat,y_stat,X,Y] = setupData(x_data,y_data,x_measure,y_measure,ravelAverageAppend,False,column_measure)

    if ravelAverageAppend == 'average':
        if isinstance(x_stat[hue_measure].unique()[0],str):
            hues = x_stat[hue_measure].unique().tolist()
        else:
            hues = x_stat.groupby(column_measure).mean()[hue_measure].tolist()
    else:
        hues = list(x_stat[hue_measure])

    if shuffleData == True:
        X,Y,hues = shuffleDataAlg(X,Y,hues)


    # generate new figure for each
    p = plt.figure()

    plt.hist2d(x=X,y=Y,cmin=1,density=False,bins=(len(X)/10),cmap='magma',vmax=(len(X)/10))
    plt.colorbar()

    # set title and x and y labels

    plt.title('%s vs %s' %(x_measure,y_measure),fontsize=20)
    plt.xlabel(x_measure,fontsize=18)
    plt.ylabel(y_measure,fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # # remove top and right spines from plot
    # p.axes.spines["top"].set_visible(False)
    # p.axes.spines["right"].set_visible(False)

    # save image or show image
    saveOrShowImg(dir_out,x_measure,y_measure,img_name)

### tract profile data
# uses matplotlib.pyplot's plot and fill_between functions to plot tract profile data from stat. useful for publication worthy figure
# requires stat to be formatted in way of AFQ_Brwoser and Yeatman et al 2018 () 'nodes.csv' files
# groups is a list array of names of groups found in 'classID' of stat to plot
# colors is a dictionary with the classID from groups set as the key and a color name as the value. will use these colors in profiles
# tracks is a list array that will be looped through to make plots. if only one track is wanted, set structures=['structure_name'], with 'structure_name' being the name of the track in the 'structureID' field of stat
# stat is the pandas dataframe with all of the profile data. each row is a node for a track for a subject
# diffusion_measures is a list array of the column measures found within stat. was developed with diffusion MRI metrics in mind, but can be any measure
# summary_method is a string of either 'mean' to plot the average profile data, 'max' to plot max, 'min' to plot min, and 'median' to plot median
# error_method is a string of either 'std' for the error bars to be set to the standard deviation or 'sem' for standard error of mean
# dir_out and imgName are the directory where the figures should be saved and the name for the image. will save .pdf and .png
# if want to view plot instead of save, set dir_out=""
def plotProfiles(structures,stat,diffusion_measures,summary_method,error_method,dir_out,img_name):

    import matplotlib.pyplot as plt
    import os,sys
    import seaborn as sns
    from scipy import stats
    import numpy as np
    import warnings
    
    warnings.filterwarnings('ignore')

    # loop through all structures
    for t in structures:
        print(t)
        # loop through all measures
        for dm in diffusion_measures:
            print(dm)

            imgname=img_name+"_"+t+"_"+dm

            # generate figures
            fig = plt.figure(figsize=(15,15))
#             fig = plt.figure()
            fig.patch.set_visible(False)
            p = plt.subplot()

            # set title and catch array for legend handle
            plt.title("%s Profiles %s: %s" %(summary_method,t,dm),fontsize=20)

            # loop through groups and plot profile data
            for g in range(len(stat.classID.unique())):
                # x is nodes
                x = stat['nodeID'].unique()

                # y is summary (mean, median, max, main) profile data
                if summary_method == 'mean':
                    y = stat[stat['classID'] == stat.classID.unique()[g]].groupby(['structureID','nodeID']).mean()[dm][t]
                elif summary_method == 'median':
                    y = stat[stat['classID'] == stat.classID.unique()[g]].groupby(['structureID','nodeID']).median()[dm][t]
                elif summary_method == 'max':
                    y = stat[stat['classID'] == stat.classID.unique()[g]].groupby(['structureID','nodeID']).max()[dm][t]
                elif summary_method == 'min':
                    y = stat[stat['classID'] == stat.classID.unique()[g]].groupby(['structureID','nodeID']).min()[dm][t]

                # error bar is either: standard error of mean (sem), standard deviation (std)
                if error_method == 'sem':
                    err = stat[stat['classID'] == stat.classID.unique()[g]].groupby(['structureID','nodeID']).std()[dm][t] / np.sqrt(len(stat[stat['classID'] == stat.classID.unique()[g]]['subjectID'].unique()))
                else:
                    err = stat[stat['classID'] == stat.classID.unique()[g]].groupby(['structureID','nodeID']).std()[dm][t]

                # plot summary
                plt.plot(x,y,color=stat[stat['classID'] == stat.classID.unique()[g]]['colors'].unique()[0],linewidth=5,label=stat.classID.unique()[g])

                # plot shaded error
                plt.fill_between(x,y-err,y+err,alpha=0.4,color=stat[stat['classID'] == stat.classID.unique()[g]]['colors'].unique()[0],label='1 %s %s' %(error_method,stat.classID.unique()[g]))
                plt.fill_between(x,y-(2*err),y+(2*err),alpha=0.2,color=stat[stat['classID'] == stat.classID.unique()[g]]['colors'].unique()[0],label='2 %s %s' %(error_method,stat.classID.unique()[g]))

            # set up labels and ticks
            plt.xlabel('Location',fontsize=18)
            plt.ylabel(dm,fontsize=18)
            plt.xticks([x[0],x[-1]],['Begin','End'],fontsize=16)
            plt.legend(fontsize=12)
            y_lim = plt.ylim()
            plt.yticks([np.round(y_lim[0],2),np.mean(y_lim),np.round(y_lim[1],2)],fontsize=16)

            # remove top and right spines from plot
            p.axes.spines["top"].set_visible(False)
            p.axes.spines["right"].set_visible(False)
            ax = plt.gca()

            # save image or show image
            saveOrShowImg(dir_out,dm,dm,imgname)

### generic data plots
## structure average
# uses matplotlib.pyplot's errobar function to plot group average data for each structure with errorbars. useful for publication worthy figure
# requires stat to be formatted in similar way of AFQ_Brwoser and Yeatman et al 2018 () 'nodes.csv' files
# groups is a list array of names of groups found in 'classID' of stat to plot
# colors is a dictionary with the classID from groups set as the key and a color name as the value. will use these colors in profiles
# tracks is a list array that will be looped through to make plots. if only one track is wanted, set structures=['structure_name'], with 'structure_name' being the name of the structure in the 'structureID' field of stat
# stat is the pandas dataframe with all of the profile data. each row is a node for a track for a subject
# diffusion_measures is a list array of the column measures found within stat. was developed with diffusion MRI metrics in mind, but can be any measure
# summary_method is a string of either 'mean' to plot the average profile data, 'max' to plot max, 'min' to plot min, and 'median' to plot median
# error_method is a string of either 'std' for the error bars to be set to the standard deviation or 'sem' for standard error of mean
# dir_out and imgName are the directory where the figures should be saved and the name for the image. will save .pdf and .png
# if want to view plot instead of save, set dir_out=""
def plotGroupStructureAverage(structures,tissue,stat,measures,summary_method,error_method,dir_out,img_name):

    import os,sys
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats

    for dm in measures:
        print(dm)

        # generate figures
        fig = plt.figure(figsize=(15,15))
        fig.patch.set_visible(False)
        p = plt.subplot()

        # set y range
        p.set_ylim([0,(len(structures)*len(stat.classID.unique()))+len(stat.classID.unique())])

        # set spines and ticks, and labels
        p.yaxis.set_ticks_position('left')
        p.xaxis.set_ticks_position('bottom')
        p.set_xlabel(dm,fontsize=18)
        p.set_ylabel("Structures",fontsize=18)
        if len(stat.classID.unique()) < 3:
            if len(stat.classID.unique()) == 2:
                p.set_yticks(np.arange(1.5,(len(structures)*len(stat.classID.unique())),step=len(stat.classID.unique())))
            else:
                p.set_yticks(np.arange(1,len(structures)+1,step=1))
        else:
            p.set_yticks(np.arange((len(stat.classID.unique())-1),(len(structures)*len(stat.classID.unique())),step=len(stat.classID.unique())))
        p.set_yticklabels(structures,fontsize=16)
        plt.xticks(fontsize=16)

        # set title
        plt.title("%s Group-Summary: %s" %(summary_method,dm),fontsize=20)

        # loop through structures
        for t in range(len(structures)):
            # loop through groups
            for g in range(len(stat.classID.unique())):
                # x is summary (mean, median, max, main) profile data
                if summary_method == 'mean':
                    x = stat[stat.classID.str.contains(stat.classID.unique()[g])].groupby(tissue).mean()[dm][structures[t]]
                elif summary_method == 'median':
                    x = stat[stat.classID.str.contains(stat.classID.unique()[g])].groupby(tissue).median()[dm][structures[t]]
                elif summary_method == 'max':
                    x = stat[stat.classID.str.contains(stat.classID.unique()[g])].groupby(tissue).max()[dm][structures[t]]
                elif summary_method == 'min':
                    x = stat[stat.classID.str.contains(stat.classID.unique()[g])].groupby(tissue).min()[dm][structures[t]]

                # y is location on y axis
                y = (len(stat.classID.unique())*(t+1)-len(stat.classID.unique()))+(g+1)

                # error bar is either: standard error of mean (sem), standard deviation (std)
                if error_method == 'sem':
                    err = stat[stat.classID.str.contains(stat.classID.unique()[g])].groupby(tissue).std()[dm][structures[t]] / np.sqrt(len(stat[stat.classID.str.contains(stat.classID.unique()[g])]['subjectID'].unique()))
                else:
                    err = stat[stat.classID.str.contains(stat.classID.unique()[g])].groupby(tissue).std()[dm][structures[t]]

                # plot data
                if t == 0:
                    p.errorbar(x=x,y=y,xerr=err,barsabove=True,ecolor='black',color=stat[stat['classID'].str.contains(stat.classID.unique()[g])]['colors'].unique()[0],marker='o',ms=10,label=stat.classID.unique()[g])
                else:
                    p.errorbar(x=x,y=y,xerr=err,barsabove=True,ecolor='black',color=stat[stat['classID'].str.contains(stat.classID.unique()[g])]['colors'].unique()[0],marker='o',ms=10)

        # add legend
        plt.legend(fontsize=16)

        # remove top and right spines from plot
        p.axes.spines["top"].set_visible(False)
        p.axes.spines["right"].set_visible(False)

        # save image or show image
        saveOrShowImg(dir_out,dm,dm,img_name)

def violinPlots(x_measure,y_measure,hue_measure,data,summary,scale,inner,cmap,dir_out,img_name):
    
    if hue_measure:
        violin = sns.violinplot(x=x_measure,y=y_measure,data=data,hue=hue_measure,scale=scale,inner=inner,palette=cmap)
    else:
        violin = sns.violinplot(x=x_measure,y=y_measure,data=data,scale=scale,inner=inner,palette=cmap,orientation='horizontal')

    if summary == 'mean':
        summary = data.groupby([x_measure])[y_measure].mean()
    elif summary == 'median':
        summary = data.groupby([x_measure])[y_measure].median()
    elif summary == 'mode':
        summary = data.groupby([x_measure])[y_measure].mode()
    elif summary == 'max':
        summary = data.groupby([x_measure])[y_measure].max()
    elif summary == 'min':
        summary = data.groupby([x_measure])[y_measure].min()

#     for xtick in violin.get_xticks():
#         violin.text(xtick,summary[xtick],np.round(summary[xtick],3),horizontalalignment='center',size='medium',color='w',weight='semibold')
  
    saveOrShowImg(dir_out,x_measure,y_measure,img_name)

In [4]:
### setting up variables and adding paths
print("setting up variables")
topPath = "./"
os.chdir(topPath)
data_dir = topPath+'/data/'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
img_dir = topPath+'/img/'
if not os.path.exists(img_dir):
    os.mkdir(img_dir)

groups = ['run_1','run_2']
colors_array = ['blue','gray']
diff_micro_measures = ['ad','fa','md','rd','ndi','isovf','odi']
diff_macro_measures = ['length','volume','count']
print("setting up variables complete")

### grabbing subjects demographic data
print("grabbing demographic data")
# if os.path.isfile('./data/subjects_data.csv'):
#     subjects_data = pd.read_csv('./data/subjects_data.csv')
# else:
subjects_data = pd.read_csv('./subjects_demo.csv')
subjects_data['subjectID'] = [ str(int(np.float(f.strip('HCP')))) for f in subjects_data['subjectID']]
subjects_data.to_csv('./data/subjects_data.csv',index=False)
# print("grabbing demographic data complete")

colors = {}
subjects = {}

# # loop through groups and identify subjects and set color schema for each group
for g in range(len(groups)):
#     # set subjects array
    subjects[groups[g]] =  subjects_data['subjectID']
    subjects[groups[g]].sort_values()
#     # update subjects with HCP prefix to make plotting easier

#     # set colors array
    colors_name = colors_array[g]
    colors[groups[g]] = colors_array[g]

### merge demo and subjects data
# subjects_data = pd.merge(subjects_data,subjects_demo,on='subjectID')
# subjects_data['classID'] = [ 'hcp' for f in range(len(subjects_data))]
# subjects_data.to_csv(data_dir+'/subjects_data_demo.csv')

# create subjects color dictionary
colors_dict = createColorDictionary(subjects_data,'subjectID','colorblind')

setting up variables
setting up variables complete
grabbing demographic data


In [7]:
subjects_data['subjectID'] =  [ str(int(np.float(f))) for f in subjects_data['subjectID']]
subjects_data.drop(columns={"Unnamed: 0"},inplace=True)

In [8]:
subjects_data

,subjectID,classID,colors,gender,age_range,age
0,103818,test,orange,F,31-35,31
1,103818,retest,blue,F,31-35,31
2,105923,test,orange,F,31-35,31
3,105923,retest,blue,F,31-35,31
4,111312,test,orange,F,31-35,31
...,...,...,...,...,...,...
83,861456,retest,blue,F,31-35,31
84,877168,test,orange,F,31-35,31
85,877168,retest,blue,F,31-35,31
86,917255,test,orange,M,31-35,31


In [16]:
subjects_data = subjects_data.loc[subjects_data['classID'] == 'test']
subjects_data.drop(columns={'classID'},inplace=True)

In [32]:
# subjects_data.drop(columns={"Unnamed: 0"},inplace=True)
# subjects_data['classID'] = [ groups[0] if f == 'test' else groups[1] for f in subjects_data['classID'] ]
subjects_data

,subjectID,classID,colors,gender,age_range,age
0,103818,run_1,orange,F,31-35,31
1,103818,run_2,blue,F,31-35,31
2,105923,run_1,orange,F,31-35,31
3,105923,run_2,blue,F,31-35,31
4,111312,run_1,orange,F,31-35,31
...,...,...,...,...,...,...
83,861456,run_2,blue,F,31-35,31
84,877168,run_1,orange,F,31-35,31
85,877168,run_2,blue,F,31-35,31
86,917255,run_1,orange,M,31-35,31


In [34]:
### app-validitiy
run_1 = collectData('neuro/tractmeasures','profiles','!validity',"output_FiberStats.csv",subjects_data,colors,data_dir+"/tract_profiles_app_validity_run1.csv")
# run_2 = collectData('neuro/tractmeasures','',['test','aparc.a2009s','bl_generated'],"tractmeasures.csv",subjects_data,colors,data_dir+"/tract_profiles_app_validity_run2.csv")


In [182]:
subjects

array(['861456', '185442', '114823', '287248', '111312', '125525',
       '137128', '137128'], dtype='<U32')

In [116]:
### freesurfer validation analysis: hcp freesurfer vs bl freesurfer
print("validating brainlife: anatomical")
# grab data: desikan killany
bl_generated_aparc_test = collectData('neuro/parc-stats','freesurfer',['test','aparc.a2009s','bl_generated'],"cortex.csv",subjects_data,colors,data_dir+"/bl_generated_freesurfer_aparc_test.csv")
hcp_provided_aparc_test = collectData('neuro/parc-stats','freesurfer',['test','aparc.a2009s','hcp_provided'],"cortex.csv",subjects_data,colors,data_dir+"/hcp_provided_freesurfer_aparc_test.csv")
bl_generated_aparc_test['classID'] = [ 'bl_generated' for f in bl_generated_aparc_test['subjectID']]
hcp_provided_aparc_test['classID'] = [ 'hcp_provided' for f in hcp_provided_aparc_test['subjectID']]
aparc_validity = pd.concat([bl_generated_aparc_test,hcp_provided_aparc_test])
aparc_validity = aparc_validity.drop_duplicates()
aparc_validity.to_csv('desikian_killany_aparc_validity_hcp_test_data.csv',index=False)

# plot data
singleplotScatter("",aparc_validity.loc[aparc_validity['classID'] == 'hcp_provided'],aparc_validity.loc[aparc_validity['classID'] == 'bl_generated'],'average_thickness_mm','average_thickness_mm',False,'structureID','structureID','ravel','linreg',True,img_dir,'aparc_validity_hcp_test_thickness')
singleplotScatter("",aparc_validity.loc[aparc_validity['classID'] == 'hcp_provided'],aparc_validity.loc[aparc_validity['classID'] == 'bl_generated'],'surface_area_mm^2','surface_area_mm^2',False,'structureID','structureID','ravel','linreg',True,img_dir,'aparc_validity_hcp_test_surface_area')
singleplotScatter("",aparc_validity.loc[aparc_validity['classID'] == 'hcp_provided'],aparc_validity.loc[aparc_validity['classID'] == 'bl_generated'],'gray_matter_volume_mm^3','gray_matter_volume_mm^3',False,'structureID','structureID','ravel','linreg',True,img_dir,'aparc_validity_hcp_test_volume')

# grab data: hcp-mmp
bl_generated_glasser_test = collectData('neuro/parc-stats','surface',['test','glasser','bl_generated'],"cortex.csv",subjects_data,colors,data_dir+"/bl_generated_freesurfer_glasser_test.csv")
hcp_provided_glasser_test = collectData('neuro/parc-stats','surface',['test','glasser','hcp_provided'],"cortex.csv",subjects_data,colors,data_dir+"/hcp_provided_freesurfer_glasser_test.csv")
bl_generated_glasser_test['classID'] = [ 'bl_generated' for f in bl_generated_glasser_test['subjectID']]
hcp_provided_glasser_test['classID'] = [ 'hcp_provided' for f in hcp_provided_glasser_test['subjectID']]

# concat and clean
glasser_validity = pd.concat([bl_generated_glasser_test,hcp_provided_glasser_test])
glasser_validity = glasser_validity[glasser_validity['structureID'] != 'lh_???']
glasser_validity = glasser_validity[glasser_validity['structureID'] != 'rh_???']
glasser_validity = glasser_validity[~glasser_validity.structureID.str.contains('unknown')]
glasser_validity = glasser_validity[~glasser_validity.structureID.str.contains('_H_ROI')]
glasser_validity = glasser_validity.drop_duplicates()
glasser_validity.to_csv('hcp_mmp_glasser_validity_hcp_test_data.csv',index=False)

# plot data
singleplotScatter("",glasser_validity.loc[glasser_validity['classID'] == 'hcp_provided'],glasser_validity.loc[glasser_validity['classID'] == 'bl_generated'],'average_thickness_mm','average_thickness_mm',False,'structureID','structureID','ravel','linreg',True,img_dir,'glasser_validity_hcp_test_thickness')
singleplotScatter("",glasser_validity.loc[glasser_validity['classID'] == 'hcp_provided'],glasser_validity.loc[glasser_validity['classID'] == 'bl_generated'],'surface_area_mm^2','surface_area_mm^2',False,'structureID','structureID','ravel','linreg',True,img_dir,'glasser_validity_hcp_test_surface_area')
singleplotScatter("",glasser_validity.loc[glasser_validity['classID'] == 'hcp_provided'],glasser_validity.loc[glasser_validity['classID'] == 'bl_generated'],'gray_matter_volume_mm^3','gray_matter_volume_mm^3',False,'structureID','structureID','ravel','linreg',True,img_dir,'glasser_validity_hcp_test_volume')

print("validating brainlife complete: anatomical")

validating brainlife: anatomical


In [131]:
### diffusion validation analysis: hcp preprocessed dwi vs bl preprocessed dwi
print("validating brainlife: diffusion")
# grab data: desikan killany
# bl_generated_aparc_test = collectData('neuro/parc-stats','freesurfer',['test','aparc.a2009s','bl_generated'],"cortex.csv",subjects_data,colors,data_dir+"/bl_generated_freesurfer_aparc_test.csv")
# hcp_provided_aparc_test = collectData('neuro/parc-stats','freesurfer',['test','aparc.a2009s','hcp_provided'],"cortex.csv",subjects_data,colors,data_dir+"/hcp_provided_freesurfer_aparc_test.csv")
# bl_generated_aparc_test['classID'] = [ 'bl_generated' for f in bl_generated_aparc_test['subjectID']]
# hcp_provided_aparc_test['classID'] = [ 'hcp_provided' for f in hcp_provided_aparc_test['subjectID']]
# aparc_validity = pd.concat([bl_generated_aparc_test,hcp_provided_aparc_test])
# aparc_validity = aparc_validity.drop_duplicates()
# aparc_validity.to_csv('desikian_killany_aparc_validity_hcp_test_data.csv',index=False)

# # plot data
# singleplotScatter("",aparc_validity.loc[aparc_validity['classID'] == 'hcp_provided'],aparc_validity.loc[aparc_validity['classID'] == 'bl_generated'],'average_thickness_mm','average_thickness_mm',False,'structureID','structureID','ravel','linreg',True,img_dir,'aparc_validity_hcp_test_thickness')
# singleplotScatter("",aparc_validity.loc[aparc_validity['classID'] == 'hcp_provided'],aparc_validity.loc[aparc_validity['classID'] == 'bl_generated'],'surface_area_mm^2','surface_area_mm^2',False,'structureID','structureID','ravel','linreg',True,img_dir,'aparc_validity_hcp_test_surface_area')
# singleplotScatter("",aparc_validity.loc[aparc_validity['classID'] == 'hcp_provided'],aparc_validity.loc[aparc_validity['classID'] == 'bl_generated'],'gray_matter_volume_mm^3','gray_matter_volume_mm^3',False,'structureID','structureID','ravel','linreg',True,img_dir,'aparc_validity_hcp_test_volume')

# # grab data: hcp-mmp
# bl_generated_glasser_test = collectData('neuro/parc-stats','surface',['test','glasser','bl_generated'],"cortex.csv",subjects_data,colors,data_dir+"/bl_generated_freesurfer_glasser_test.csv")
# hcp_provided_glasser_test = collectData('neuro/parc-stats','surface',['test','glasser','hcp_provided'],"cortex.csv",subjects_data,colors,data_dir+"/hcp_provided_freesurfer_glasser_test.csv")
# bl_generated_glasser_test['classID'] = [ 'bl_generated' for f in bl_generated_glasser_test['subjectID']]
# hcp_provided_glasser_test['classID'] = [ 'hcp_provided' for f in hcp_provided_glasser_test['subjectID']]

# # concat and clean
# glasser_validity = pd.concat([bl_generated_glasser_test,hcp_provided_glasser_test])
# glasser_validity = glasser_validity[glasser_validity['structureID'] != 'lh_???']
# glasser_validity = glasser_validity[glasser_validity['structureID'] != 'rh_???']
# glasser_validity = glasser_validity[~glasser_validity.structureID.str.contains('unknown')]
# glasser_validity = glasser_validity[~glasser_validity.structureID.str.contains('_H_ROI')]
# glasser_validity = glasser_validity.drop_duplicates()
# glasser_validity.to_csv('hcp_mmp_glasser_validity_hcp_test_data.csv',index=False)

# # plot data
# singleplotScatter("",glasser_validity.loc[glasser_validity['classID'] == 'hcp_provided'],glasser_validity.loc[glasser_validity['classID'] == 'bl_generated'],'average_thickness_mm','average_thickness_mm',False,'structureID','structureID','ravel','linreg',True,img_dir,'glasser_validity_hcp_test_thickness')
# singleplotScatter("",glasser_validity.loc[glasser_validity['classID'] == 'hcp_provided'],glasser_validity.loc[glasser_validity['classID'] == 'bl_generated'],'surface_area_mm^2','surface_area_mm^2',False,'structureID','structureID','ravel','linreg',True,img_dir,'glasser_validity_hcp_test_surface_area')
# singleplotScatter("",glasser_validity.loc[glasser_validity['classID'] == 'hcp_provided'],glasser_validity.loc[glasser_validity['classID'] == 'bl_generated'],'gray_matter_volume_mm^3','gray_matter_volume_mm^3',False,'structureID','structureID','ravel','linreg',True,img_dir,'glasser_validity_hcp_test_volume')

print("validating brainlife complete: diffusion")

In [ ]:
### diffusion validation analysis: hcp preprocessed fmri vs bl preprocessed fmri
print("validating brainlife: fmri")
# grab data: desikan killany
# bl_generated_aparc_test = collectData('neuro/parc-stats','freesurfer',['test','aparc.a2009s','bl_generated'],"cortex.csv",subjects_data,colors,data_dir+"/bl_generated_freesurfer_aparc_test.csv")
# hcp_provided_aparc_test = collectData('neuro/parc-stats','freesurfer',['test','aparc.a2009s','hcp_provided'],"cortex.csv",subjects_data,colors,data_dir+"/hcp_provided_freesurfer_aparc_test.csv")
# bl_generated_aparc_test['classID'] = [ 'bl_generated' for f in bl_generated_aparc_test['subjectID']]
# hcp_provided_aparc_test['classID'] = [ 'hcp_provided' for f in hcp_provided_aparc_test['subjectID']]
# aparc_validity = pd.concat([bl_generated_aparc_test,hcp_provided_aparc_test])
# aparc_validity = aparc_validity.drop_duplicates()
# aparc_validity.to_csv('desikian_killany_aparc_validity_hcp_test_data.csv',index=False)

# # plot data
# singleplotScatter("",aparc_validity.loc[aparc_validity['classID'] == 'hcp_provided'],aparc_validity.loc[aparc_validity['classID'] == 'bl_generated'],'average_thickness_mm','average_thickness_mm',False,'structureID','structureID','ravel','linreg',True,img_dir,'aparc_validity_hcp_test_thickness')
# singleplotScatter("",aparc_validity.loc[aparc_validity['classID'] == 'hcp_provided'],aparc_validity.loc[aparc_validity['classID'] == 'bl_generated'],'surface_area_mm^2','surface_area_mm^2',False,'structureID','structureID','ravel','linreg',True,img_dir,'aparc_validity_hcp_test_surface_area')
# singleplotScatter("",aparc_validity.loc[aparc_validity['classID'] == 'hcp_provided'],aparc_validity.loc[aparc_validity['classID'] == 'bl_generated'],'gray_matter_volume_mm^3','gray_matter_volume_mm^3',False,'structureID','structureID','ravel','linreg',True,img_dir,'aparc_validity_hcp_test_volume')

# # grab data: hcp-mmp
# bl_generated_glasser_test = collectData('neuro/parc-stats','surface',['test','glasser','bl_generated'],"cortex.csv",subjects_data,colors,data_dir+"/bl_generated_freesurfer_glasser_test.csv")
# hcp_provided_glasser_test = collectData('neuro/parc-stats','surface',['test','glasser','hcp_provided'],"cortex.csv",subjects_data,colors,data_dir+"/hcp_provided_freesurfer_glasser_test.csv")
# bl_generated_glasser_test['classID'] = [ 'bl_generated' for f in bl_generated_glasser_test['subjectID']]
# hcp_provided_glasser_test['classID'] = [ 'hcp_provided' for f in hcp_provided_glasser_test['subjectID']]

# # concat and clean
# glasser_validity = pd.concat([bl_generated_glasser_test,hcp_provided_glasser_test])
# glasser_validity = glasser_validity[glasser_validity['structureID'] != 'lh_???']
# glasser_validity = glasser_validity[glasser_validity['structureID'] != 'rh_???']
# glasser_validity = glasser_validity[~glasser_validity.structureID.str.contains('unknown')]
# glasser_validity = glasser_validity[~glasser_validity.structureID.str.contains('_H_ROI')]
# glasser_validity = glasser_validity.drop_duplicates()
# glasser_validity.to_csv('hcp_mmp_glasser_validity_hcp_test_data.csv',index=False)

# # plot data
# singleplotScatter("",glasser_validity.loc[glasser_validity['classID'] == 'hcp_provided'],glasser_validity.loc[glasser_validity['classID'] == 'bl_generated'],'average_thickness_mm','average_thickness_mm',False,'structureID','structureID','ravel','linreg',True,img_dir,'glasser_validity_hcp_test_thickness')
# singleplotScatter("",glasser_validity.loc[glasser_validity['classID'] == 'hcp_provided'],glasser_validity.loc[glasser_validity['classID'] == 'bl_generated'],'surface_area_mm^2','surface_area_mm^2',False,'structureID','structureID','ravel','linreg',True,img_dir,'glasser_validity_hcp_test_surface_area')
# singleplotScatter("",glasser_validity.loc[glasser_validity['classID'] == 'hcp_provided'],glasser_validity.loc[glasser_validity['classID'] == 'bl_generated'],'gray_matter_volume_mm^3','gray_matter_volume_mm^3',False,'structureID','structureID','ravel','linreg',True,img_dir,'glasser_validity_hcp_test_volume')

print("validating brainlife complete: fmri")

In [7]:
### cortical data reliability: test-retest with hcp-freesurfer
print("grabbing aparc a2009s data")

### freesurfer annotation statistics - bl generated
print("grabbing bl generated aparc a2009s data")
# grab data
aparc_stats_bl = collectData(topPath,"parc-freesurfer-bl","cortex.csv",data_dir,groups,subjects,'generated-a2009s')

# plot data
for measures in structural_measures:
    print(measures)
    singleplotScatter(colors_dict,aparc_stats_bl[aparc_stats_bl['classID'] == groups[0]],aparc_stats_bl[aparc_stats_bl['classID'] == groups[1]],measures,measures,'structureID','subjectID','ravel','linreg',True,img_dir,"aparc_stats_bl_scatter")

### glasser annotation statistics
print("grabbing glasser data")
# grab data
glasser_stats_bl = collectData(topPath,"parc-glasser-bl","cortex.csv",data_dir,groups,subjects,'bl-generated')

# clean up data
glasser_stats_bl = glasser_stats_bl[glasser_stats_bl['structureID'] != 'lh_???']
glasser_stats_bl = glasser_stats_bl[glasser_stats_bl['structureID'] != 'rh_???'] 
glasser_stats_bl = glasser_stats_bl[~glasser_stats_bl.structureID.str.contains('unknown')]
structureList = []
for i in glasser_stats_bl['structureID'].unique():
    if len(glasser_stats_bl[glasser_stats_bl['structureID'] == i].subjectID) < len(groups) * len(glasser_stats_bl['subjectID'].unique()):
        print('%s is missing in all subjects' %i)
    else:
        structureList = np.append(structureList,i)
glasser_stats_bl = glasser_stats_bl.loc[glasser_stats_bl['structureID'].isin(structureList)]
glasser_stats_bl.to_csv(data_dir+'/parc-glasser-bl-cleaned.csv')

# plot data
for measures in structural_measures:
    print(measures)
    singleplotScatter(colors_dict,glasser_stats_bl[glasser_stats_bl['classID'] == groups[0]],glasser_stats_bl[glasser_stats_bl['classID'] == groups[1]],measures,measures,'structureID','subjectID','ravel','linreg',True,img_dir,"glasser_stats_bl_scatter")


,subjectID,classID,colors,gender,age_range,age
0,103818,test,orange,F,31-35,31
1,103818,retest,blue,F,31-35,31
2,105923,test,orange,F,31-35,31
3,105923,retest,blue,F,31-35,31
4,111312,test,orange,F,31-35,31
...,...,...,...,...,...,...
83,861456,retest,blue,F,31-35,31
84,877168,test,orange,F,31-35,31
85,877168,retest,blue,F,31-35,31
86,917255,test,orange,M,31-35,31


In [101]:
# fmri validity
bl_generated_fmri = collectData('generic/network','measurements',['test','bl_generated'],'network.json.gz',subjects_data,colors,data_dir+"/bl_generated_fmri_yeo_validity_test.csv")
bl_generated_fmri.drop(columns={'colors'},inplace=True)
bl_generated_fmri.drop_duplicates('subjectID',inplace=True)

hcp_provided_fmri = collectData('generic/network','measurements',['test','bl'],'network.json.gz',subjects_data,colors,data_dir+"/hcp_provided_fmri_yeo_validity_test.csv")
hcp_provided_fmri['classID'] = ['run_2' for f in hcp_provided_fmri['classID']]
hcp_provided_fmri.drop(columns={'colors'},inplace=True)
hcp_provided_fmri.drop_duplicates('subjectID',inplace=True)

# bl_generated_fmri_avg_degree = [ bl_generated_fmri.loc[bl_generated_fmri['subjectID'] == f]['metadata'].reset_index(drop=True)[0]['Avg. Degree'] for f in bl_generated_fmri['subjectID'].unique()] 
# hcp_provided_fmri_avg_degree = [ hcp_provided_fmri.loc[hcp_provided_fmri['subjectID'] == f]['metadata'].reset_index(drop=True)[0]['Avg. Degree'] for f in hcp_provided_fmri['subjectID'].unique()] 

# fmri_average_degree = pd.DataFrame()
# fmri_average_degree['subjectID'] = bl_generated_fmri.subjectID.unique().tolist() + hcp_provided_fmri.subjectID.unique().tolist()
# fmri_average_degree['classID'] = bl_generated_fmri.classID.tolist() + hcp_provided_fmri.classID.tolist()

# fmri_average_degree['degree'] = bl_generated_fmri_avg_degree + hcp_provided_fmri_avg_degree
# singleplotScatter('',fmri_average_degree.loc[fmri_average_degree['classID'] == 'run_1'],fmri_average_degree.loc[fmri_average_degree['classID'] == 'run_2'],'degree','degree',False,'classID','classID','ravel','linreg',True,img_dir,'bl_fmri_average_degree_validity')


In [93]:
node_degree = []
for i in hcp_provided_fmri['subjectID']:
    data = hcp_provided_fmri.loc[hcp_provided_fmri['subjectID'] == i]
    
hcp_provided_fmri

,label,directed,type,metadata,nodes,edges,subjectID,classID,colors,gender,age_range,age
0,network,False,correlation,"{'Avg. Degree': 16.307692307692307, 'Avg. Stre...","{'0': {'label': 'ROI_1', 'metadata': {'column_...","[{'source': '0', 'target': '10', 'metadata': {...",103818,run_2,orange,F,31-35,31
1,network,False,correlation,"{'Avg. Degree': 16.307692307692307, 'Avg. Stre...","{'0': {'label': 'ROI_1', 'metadata': {'column_...","[{'source': '0', 'target': '10', 'metadata': {...",103818,run_2,blue,F,31-35,31
2,network,False,correlation,"{'Avg. Degree': 18.16923076923077, 'Avg. Stren...","{'0': {'label': 'ROI_1', 'metadata': {'column_...","[{'source': '0', 'target': '29', 'metadata': {...",105923,run_2,orange,F,31-35,31
3,network,False,correlation,"{'Avg. Degree': 18.16923076923077, 'Avg. Stren...","{'0': {'label': 'ROI_1', 'metadata': {'column_...","[{'source': '0', 'target': '29', 'metadata': {...",105923,run_2,blue,F,31-35,31
4,network,False,correlation,"{'Avg. Degree': 15.6, 'Avg. Strength': 6.04927...","{'0': {'label': 'ROI_1', 'metadata': {'column_...","[{'source': '0', 'target': '47', 'metadata': {...",111312,run_2,orange,F,31-35,31
...,...,...,...,...,...,...,...,...,...,...,...,...
59,network,False,correlation,"{'Avg. Degree': 7.430769230769231, 'Avg. Stren...","{'0': {'label': 'ROI_1', 'metadata': {'column_...","[{'source': '0', 'target': '77', 'metadata': {...",204521,run_2,blue,F,31-35,31
60,network,False,correlation,"{'Avg. Degree': 18.076923076923077, 'Avg. Stre...","{'0': {'label': 'ROI_1', 'metadata': {'column_...","[{'source': '0', 'target': '28', 'metadata': {...",250427,run_2,orange,F,31-35,31
61,network,False,correlation,"{'Avg. Degree': 18.076923076923077, 'Avg. Stre...","{'0': {'label': 'ROI_1', 'metadata': {'column_...","[{'source': '0', 'target': '28', 'metadata': {...",250427,run_2,blue,F,31-35,31
62,network,False,correlation,"{'Avg. Degree': 14.415384615384616, 'Avg. Stre...","{'0': {'label': 'ROI_1', 'metadata': {'column_...","[{'source': '0', 'target': '11', 'metadata': {...",287248,run_2,orange,F,26-30,26


In [171]:
# hcp_provided_fmri.drop(columns={'colors'},inplace=True)
hcp_provided_fmri_node_degree = []
hcp_provided_fmri_subjects = []
for i in hcp_provided_fmri.subjectID.unique():
    for j in hcp_provided_fmri.loc[hcp_provided_fmri['subjectID'] == i]['nodes'].reset_index(drop=True)[0]:
        hcp_provided_fmri_subjects = np.append(hcp_provided_fmri_subjects,i)
        hcp_provided_fmri_node_degree = np.append(hcp_provided_fmri_node_degree,hcp_provided_fmri.loc[hcp_provided_fmri['subjectID'] == i]['nodes'].reset_index(drop=True)[0][j]['metadata']['Degree'])
        
bl_generated_fmri_node_degree = []
bl_generated_fmri_subjects = []
for i in bl_generated_fmri.subjectID.unique():
    for j in bl_generated_fmri.loc[bl_generated_fmri['subjectID'] == i]['nodes'].reset_index(drop=True)[0]:
        bl_generated_fmri_subjects = np.append(bl_generated_fmri_subjects,i)
        bl_generated_fmri_node_degree = np.append(bl_generated_fmri_node_degree,bl_generated_fmri.loc[bl_generated_fmri['subjectID'] == i]['nodes'].reset_index(drop=True)[0][j]['metadata']['Degree'])
        
node_degree_validity = pd.DataFrame()
node_degree_validity['degree'] = np.append(hcp_provided_fmri_node_degree,bl_generated_fmri_node_degree)
node_degree_validity['classID'] = np.append([ 'hcp_provided' for f in hcp_provided_fmri_node_degree],[ 'bl_generated' for f in bl_generated_fmri_node_degree])
node_degree_validity['subjectID'] = np.append(hcp_provided_fmri_subjects,bl_generated_fmri_subjects)

In [178]:
singleplotScatter('',node_degree_validity.loc[node_degree_validity['classID'] == 'hcp_provided'],node_degree_validity.loc[node_degree_validity['classID'] == 'bl_generated'],'degree','degree',False,'subjectID','subjectID','ravel','linreg',True,img_dir,'fmri_yeo_validity_node_degree')


In [31]:
subjects_data['classID'] = [ 'run_1' if f == 'test' else 'run_2' for f in subjects_data['classID'] ]
# tract_profiles_run_1 = collectData('neuro/tractmeasures','profiles',['validity','!run_2'],'output_FiberStats.csv',subjects_data,colors,data_dir+"/tract_profiles_within_app_validity_run_1.csv")
tract_profiles_run_1['classID'] = [ 'run_1' for f in tract_profiles_run_1['subjectID'] ]
# tract_profiles_run_2 = collectData('neuro/tractmeasures','profiles',['validity','run_2'],'output_FiberStats.csv',subjects_data,colors,data_dir+"/tract_profiles_within_app_validity_run_2.csv")
tract_profiles_run_2['classID'] = [ 'run_2' for f in tract_profiles_run_2['subjectID'] ]



In [86]:
tract_profiles = pd.concat([tract_profiles_run_1,tract_profiles_run_2])

In [87]:

tract_profiles_average = computeMeanData(data_dir,tract_profiles,'tract_profiles_within_app_validity_average')

In [90]:
singleplotScatter('',tract_profiles_average.loc[tract_profiles_average['classID'] == 'run_1'],tract_profiles_average.loc[tract_profiles_average['classID'] == 'run_2'],'fa','fa',False,'structureID','subjectID','ravel','linreg',True,img_dir,'tract_profiles_within_app_validity')


In [ ]:
sns.scatterplot(x=tract_profiles_average,y=tract_profiles_run_2['fa'].values,hue=tract_profiles_run_1['structureID'].values,legend=False)